In [1]:
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime
import cv2


In [2]:
Path_To_Raw_Data = '/Users/sthitapati/Library/CloudStorage/OneDrive-UniversityCollegeLondon/tracking_analysis/data/bpod_raw_data'
output_folder = '/Users/sthitapati/Library/CloudStorage/OneDrive-UniversityCollegeLondon/tracking_analysis/data'
camera_folder = '/Users/sthitapati/Library/CloudStorage/OneDrive-UniversityCollegeLondon/tracking_analysis/data/SP_FlyCap'

In [3]:
Animal_ID = 'SP151'

In [4]:
# load files like /Users/sthitapati/Library/CloudStorage/OneDrive-UniversityCollegeLondon/tracking_analysis/data/SP151/SP151_transition_data_all_sessions.csv

transition_file_format = Animal_ID + '_transition_data_all_sessions.csv'

transition_data_file = os.path.join(output_folder, Animal_ID, transition_file_format)
print(transition_data_file)


/Users/sthitapati/Library/CloudStorage/OneDrive-UniversityCollegeLondon/tracking_analysis/data/SP151/SP151_transition_data_all_sessions.csv


In [5]:
# load transition data

transition_data = pd.read_csv(transition_data_file)
transition_data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sthitapati/Library/CloudStorage/OneDrive-UniversityCollegeLondon/tracking_analysis/data/SP151/SP151_transition_data_all_sessions.csv'

In [ ]:
# load files with format /Users/sthitapati/Library/CloudStorage/OneDrive-UniversityCollegeLondon/tracking_analysis/data/SP151/SP151_session_info.csv

session_info_file_format = Animal_ID + '_session_info.csv'

session_info_file = os.path.join(output_folder, Animal_ID, session_info_file_format)
print(session_info_file)

/Users/sthitapati/Library/CloudStorage/OneDrive-UniversityCollegeLondon/tracking_analysis/data/SP151/SP151_session_info.csv


In [ ]:
# load session info

session_info = pd.read_csv(session_info_file)
session_info.head()

,session_id,session,date,file_number,day,opto_session,stim_port,opto_chance,pulse_duration,pulse_interval,train_duration,train_delay,variable_train_delay,mu_variable_delay,sigma_variable_delay,lower_bound_variableDelay,upper_bound_variableDelay,experiment_type
0,0,00_20231120_141633_Mon,2023-11-20,141633,Mon,False,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,1_Training
1,1,01_20231121_133350_Tue,2023-11-21,133350,Tue,False,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,1_Training
2,2,02_20231122_133348_Wed,2023-11-22,133348,Wed,False,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,1_Training
3,3,03_20231123_150015_Thu,2023-11-23,150015,Thu,False,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,1_Training
4,4,04_20231124_180411_Fri,2023-11-24,180411,Fri,False,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,1_Training


In [ ]:
# get all session dates and convert to datetime
all_session_dates = pd.to_datetime(session_info['date'], format='%Y-%m-%d')


In [ ]:

current_camera_folder = os.path.join(camera_folder, Animal_ID)  
DLC_files_pattern = 'DLC_resnet50_seq_8_keypoints_thethredFeb27shuffle1_100000'
# list all avi files in the current camera folder 
all_files = os.listdir(current_camera_folder)
print(all_files)
avi_files = [f for f in all_files if f.endswith('.avi')]
print(avi_files)
stamp_files = [f for f in all_files if f.startswith('stamps')]
print(stamp_files)
dlc_h5_files = [f for f in all_files if f.endswith('.h5') and DLC_files_pattern in f]  # deep lab cut files
print(dlc_h5_files)
dlc_pickle_metadata_files = [f for f in all_files if f.endswith('.pickle') and DLC_files_pattern in f]  # deep lab cut metadata files
print(dlc_pickle_metadata_files)
 # deep lab cut csv files that has DLC_files_pattern in the name
dlc_csv_files = [f for f in all_files if f.endswith('.csv') and DLC_files_pattern in f]
print(dlc_csv_files)


['camera12024-02-13T11_11_42.mp4', 'camera12024-02-13T11_11_42DLC_resnet50_seq_8_keypoints_thethredFeb27shuffle1_100000_meta.pickle', 'camera12024-02-13T11_11_42DLC_resnet50_seq_8_keypoints_thethredFeb27shuffle1_100000.csv', 'stamps2024-02-13T11_11_42.csv', 'camera12024-02-13T11_11_42DLC_resnet50_seq_8_keypoints_thethredFeb27shuffle1_100000.h5', 'camera12024-02-13T11_11_42.avi']
['camera12024-02-13T11_11_42.avi']
['stamps2024-02-13T11_11_42.csv']
['camera12024-02-13T11_11_42DLC_resnet50_seq_8_keypoints_thethredFeb27shuffle1_100000.h5']
['camera12024-02-13T11_11_42DLC_resnet50_seq_8_keypoints_thethredFeb27shuffle1_100000_meta.pickle']
['camera12024-02-13T11_11_42DLC_resnet50_seq_8_keypoints_thethredFeb27shuffle1_100000.csv']


In [ ]:
def get_date_from_filename(filename):
    """
    Extracts the date from the filename using regular expression.
    
    The function expects the date in the format YYYY-MM-DD within the filename.
    
    Args:
    filename (str): The name of the file from which to extract the date.
    
    Returns:
    str: The extracted date in the format YYYY-MM-DD, or None if no date is found.
    """
    # Regular expression pattern for matching dates in the format YYYY-MM-DD
    date_pattern = r'\d{4}-\d{2}-\d{2}'
    
    # Search for the pattern in the filename
    match = re.search(date_pattern, filename)
    
    # If a match is found, return the matched date string, else return None
    if match:
        return match.group()
    else:
        return None


for filename in all_files:
    print(f"{filename} -> Date: {get_date_from_filename(filename)}")

camera12024-02-13T11_11_42.mp4 -> Date: 2024-02-13
camera12024-02-13T11_11_42DLC_resnet50_seq_8_keypoints_thethredFeb27shuffle1_100000_meta.pickle -> Date: 2024-02-13
camera12024-02-13T11_11_42DLC_resnet50_seq_8_keypoints_thethredFeb27shuffle1_100000.csv -> Date: 2024-02-13
stamps2024-02-13T11_11_42.csv -> Date: 2024-02-13
camera12024-02-13T11_11_42DLC_resnet50_seq_8_keypoints_thethredFeb27shuffle1_100000.h5 -> Date: 2024-02-13
camera12024-02-13T11_11_42.avi -> Date: 2024-02-13


In [ ]:
# find session_infor.session_id for which opto_session is True

opto_sessions = session_info[session_info['opto_session'] == True]
# opto_sessions

In [ ]:
opto_df = transition_data[transition_data['session_id'].isin(opto_sessions['session_id'])]


In [ ]:
# load the first trial of opto_df

# unique_cumulative_trial_ids = opto_df['cumulative_trial_id'].unique()


# trial_df = opto_df[opto_df['cumulative_trial_id'] == unique_cumulative_trial_ids[0]]

In [ ]:
# we have video data for 2024-02-13
#  fetch opto_df for 2024-02-13

trial_df = opto_df[opto_df['date'] == '2024-02-13']

In [ ]:
trial_df

,Unnamed: 0,trial_id,transition_type,start_poke_port,end_poke_port,start_poke_in_timestamp,start_poke_out_timestamp,end_poke_in_timestamp,end_poke_out_timestamp,out_in_latency,...,camera_port_in_times,camera_port_out_times,camera_trial_start_times,camera_trial_end_times,cumulative_trial_id,session_id,date,day,animal_id,group
227766,0,1,43,4,3,70.9440,71.0146,71.3283,71.3297,0.3137,...,179.956625,180.027225,181.014625,189.027000,27241,67,2024-02-13,Tue,SP151,Control
227767,1,1,32,3,2,71.3283,71.3297,72.0020,72.3835,0.6723,...,180.340925,180.342325,181.014625,189.027000,27241,67,2024-02-13,Tue,SP151,Control
227768,2,1,21,2,1,72.0020,72.3835,72.6204,72.6811,0.2369,...,181.014625,181.396125,181.014625,189.027000,27241,67,2024-02-13,Tue,SP151,Control
227769,3,1,16,1,6,72.6204,72.6811,73.0352,73.1049,0.3541,...,181.633025,181.693725,181.014625,189.027000,27241,67,2024-02-13,Tue,SP151,Control
227770,4,1,67,6,7,73.0352,73.1049,73.5079,73.5532,0.4030,...,182.047825,182.117525,181.014625,189.027000,27241,67,2024-02-13,Tue,SP151,Control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232119,4353,513,37,3,7,3958.5265,3958.5510,3964.2863,3964.4008,5.7353,...,4067.658550,4067.683050,4121.315250,4126.712375,27753,67,2024-02-13,Tue,SP151,Control
232120,4354,513,72,7,2,3964.2863,3964.4008,4012.1832,4012.1833,47.7824,...,4073.418350,4073.532850,4121.315250,4126.712375,27753,67,2024-02-13,Tue,SP151,Control
232121,4355,513,22,2,2,4012.1832,4012.1833,4012.1834,4012.2336,0.0001,...,4121.315250,4121.315350,4121.315250,4126.712375,27753,67,2024-02-13,Tue,SP151,Control
232122,4356,513,21,2,1,4012.1834,4012.2336,4056.1397,4056.8501,43.9061,...,4121.315450,4121.365650,4121.315250,4126.712375,27753,67,2024-02-13,Tue,SP151,Control


In [ ]:
# now exctract some optotrials, I know for example that trial_id = 4 is an optotrial

opto_trial = trial_df[trial_df['trial_id'] == 4]

opto_trial

,Unnamed: 0,trial_id,transition_type,start_poke_port,end_poke_port,start_poke_in_timestamp,start_poke_out_timestamp,end_poke_in_timestamp,end_poke_out_timestamp,out_in_latency,...,camera_port_in_times,camera_port_out_times,camera_trial_start_times,camera_trial_end_times,cumulative_trial_id,session_id,date,day,animal_id,group
227795,29,4,26,2,6,96.9284,97.0033,101.9534,101.9838,4.9501,...,205.93475,206.00965,205.93475,211.315125,27244,67,2024-02-13,Tue,SP151,Control
227796,30,4,66,6,6,101.9534,101.9838,102.0127,102.0219,0.0289,...,210.95975,210.99015,205.93475,211.315125,27244,67,2024-02-13,Tue,SP151,Control
227797,31,4,66,6,6,102.0127,102.0219,102.1519,102.2657,0.1300,...,211.01905,211.02825,205.93475,211.315125,27244,67,2024-02-13,Tue,SP151,Control
227798,32,4,62,6,2,102.1519,102.2657,102.6411,102.6412,0.3754,...,211.15825,211.27205,205.93475,211.315125,27244,67,2024-02-13,Tue,SP151,Control


In [ ]:
# find the camera_trial_start_timestamp and camera_trial_end_timestamp for the first trial

camera_trial_start_timestamp = opto_trial['camera_trial_start_times'].values[0]
camera_trial_end_timestamp = opto_trial['camera_trial_end_times'].values[0]

print(camera_trial_start_timestamp, camera_trial_end_timestamp)


205.93475 211.315125


In [ ]:
# load video from the date of the trial

# get the date of the trial
trial_date = trial_df['date'].values[0]
print(trial_date)

# get the video file for the date of the trial
# loop though files in all_files, find the date and match with the trial_date
# if the date matches, load the video file with avi extension

video_file = None
for filename in all_files:
    date = get_date_from_filename(filename)
    if date == trial_date and filename.endswith('.avi'):
        print(f"Found video file for date {date}: {filename}")
        video_file = filename
        break

2024-02-13
Found video file for date 2024-02-13: camera12024-02-13T11_11_42.avi


In [ ]:
# load the video file and extract the frames between camera_trial_start_timestamp and camera_trial_end_timestamp

# load the video file
video_file_path = os.path.join(current_camera_folder, video_file)
# check if the file exists
if os.path.exists(video_file_path):
    print(f"Loading video file: {video_file_path}")

# load the video file
video = cv2.VideoCapture(video_file_path)



Loading video file: /Users/sthitapati/Library/CloudStorage/OneDrive-UniversityCollegeLondon/tracking_analysis/data/SP_FlyCap/SP151/camera12024-02-13T11_11_42.avi


In [ ]:
# get the frame rate of the video
frame_rate = video.get(cv2.CAP_PROP_FPS)
print(f"Frame rate of the video: {frame_rate}")

# get the total number of frames in the video
total_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)

# get the width and height of the video
width = video.get(cv2.CAP_PROP_FRAME_WIDTH)

height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)

print(f"Total frames: {total_frames}, Width: {width}, Height: {height}")

Frame rate of the video: 60.0
Total frames: 246001.0, Width: 1280.0, Height: 960.0


In [ ]:
# get the size of the video in GB

video_size_gb = os.path.getsize(video_file_path) / (1024 ** 3)
print(f"Size of the video: {video_size_gb:.2f} GB")

Size of the video: 3.78 GB


In [ ]:

# os.makedirs(output_folder, exist_ok=True)

# # Get the frame rate of the video
# frame_rate = video.get(cv2.CAP_PROP_FPS)

# # Convert timestamps to milliseconds
# start_time_ms = camera_trial_start_timestamp * 1000
# end_time_ms = camera_trial_end_timestamp * 1000

# # Set the video to the start of the trial
# video.set(cv2.CAP_PROP_POS_MSEC, start_time_ms)

# # Read the first frame
# success, frame = video.read()

# # see the first frame
# cv2.imshow('First Frame', frame)
# cv2.waitKey(0) # wait for any key to be pressed
# cv2.destroyAllWindows()

# # Release the video
# video.release()
